In [1]:
import requests
import html5lib
import time
from bs4 import BeautifulSoup
import pandas as pd

standingsURL = 'https://fbref.com/en/comps/9/premier-league-stats'

years = list(range(2022, 2015, -1))

allMatches = []

for year in years:
    data = requests.get(standingsURL)
    soup = BeautifulSoup(data.text)
    # time.sleep(5)
    # standingsTable
    standingsTable = soup.select('table.stats_table')[0]

    links = [l.get('href') for l in standingsTable.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    teamURLs = [f'https://fbref.com{l}' for l in links]
    # time.sleep(5)
    data = requests.get(standingsURL)
    previousSeasons = soup.select('a.button2.prev')[0].get('href')
    standingsURL = f'https://fbref.com{previousSeasons}'

    data = requests.get(teamURLs[0])

    for teamURL in teamURLs:
        links = standingsTable.find_all('a')
        links = [l.get('href') for l in links]
        links = [l for l in links if '/squads/' in l]
        teamURLs = [f'https://fbref.com{l}' for l in links]
        teamName = teamURL.split('/')[-1].replace('-Stats', '').replace('-', ' ')

        data = requests.get(teamURL)
        matches = pd.read_html(data.text, match='Scores & Fixtures')[0]

        soup = BeautifulSoup(data.text)
        # time.sleep(5)
        links = [l.get('href') for l in soup.find_all('a')]
        print(links)
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f'https://fbref.com{links[0]}')
        shooting = pd.read_html(data.text, match='Shooting')[0]
        shooting.columns = shooting.columns.droplevel()

        try:
            teamData = matches.merge(shooting[['Date', 'Sh', 'SoT', 'Dist', 'FK', 'PK', 'PKatt']], on='Date')
        except ValueError:
            continue

        teamData = teamData[teamData['Comp'] == 'Premier League']
        teamData['Season'] = year
        teamData['Team'] = teamName
        allMatches.append(teamData)
        time.sleep(5)

matchDF = pd.concat(allMatches)

matchDF.columns = [c.lower() for c in matchDF.columns]

matchDF.to_csv('myMatches.csv')

ValueError: No tables found